In [9]:
import numpy as np
Y= np.array([1,0,0,1,0,1,0,1])
X= np.array([1,1,4,1,1,1,3,2])

values, counts= np.unique(Y,return_counts= True)

# print(values)
# print(counts)

def entropy(var):
    N= var.shape[0]  # return total elements present
    values, counts= np.unique(var,return_counts= True)

    ent=0.0

    for i in counts:
        p= i/N
        ent+= (p*np.log2(p))
    return -ent  

entropy(X)      


[0 1]
[4 4]


1.5487949406953985

In [58]:
# CODE SPLIT
import pandas as pd
data= pd.read_csv('golf_df.csv')
def divide_data(data,feature):
    DATA= {}
    feat_values= list(data[feature].value_counts().index)
    # unique values --> print(feat_values)

    occurances= list(data[feature].value_counts())
    # unique values numbers --> print(occurances)

    for val in feat_values:
        DATA[val]= {'data': pd.DataFrame([], columns= data.columns), 'len' : 0}

    # print(DATA )   
    # prints total rows --> print(data.shape[0])

    for ix in range(data.shape[0]):
        val= data[feature].iloc[ix]

        # print(val) 

        DATA[val]['data'] = pd.concat([DATA[val]['data'], data.iloc[[ix]]], ignore_index=True)


        idx= feat_values.index(val) 
        # To get index in feat_values --> print(idx)
        DATA[val]['len']= occurances[idx]
    return DATA  

divide_data(data, 'Outlook')  

{'sunny': {'data':   Outlook Temperature Humidity  Windy Play
  0   sunny         hot     high  False   no
  1   sunny         hot     high   True   no
  2   sunny        mild     high  False   no
  3   sunny        cool   normal  False  yes
  4   sunny        mild   normal   True  yes,
  'len': 5},
 'rainy': {'data':   Outlook Temperature Humidity  Windy Play
  0   rainy        mild     high  False  yes
  1   rainy        cool   normal  False  yes
  2   rainy        cool   normal   True   no
  3   rainy        mild   normal  False  yes
  4   rainy        mild     high   True   no,
  'len': 5},
 'overcast': {'data':     Outlook Temperature Humidity  Windy Play
  0  overcast         hot     high  False  yes
  1  overcast        cool   normal   True  yes
  2  overcast        mild     high   True  yes
  3  overcast         hot   normal  False  yes,
  'len': 4}}

In [113]:
for i in range(14):
   print(data.iloc[i])

Outlook            2
Temperature        1
Humidity           0
Windy          False
Play               0
Name: 0, dtype: object
Outlook           2
Temperature       1
Humidity          0
Windy          True
Play              0
Name: 1, dtype: object
Outlook            0
Temperature        1
Humidity           0
Windy          False
Play               1
Name: 2, dtype: object
Outlook            1
Temperature        2
Humidity           0
Windy          False
Play               1
Name: 3, dtype: object
Outlook            1
Temperature        0
Humidity           1
Windy          False
Play               1
Name: 4, dtype: object
Outlook           1
Temperature       0
Humidity          1
Windy          True
Play              0
Name: 5, dtype: object
Outlook           0
Temperature       0
Humidity          1
Windy          True
Play              1
Name: 6, dtype: object
Outlook            2
Temperature        2
Humidity           0
Windy          False
Play               0
Name: 7, dtype

In [103]:
data['Outlook'].value_counts()

Outlook
2    5
1    5
0    4
Name: count, dtype: int64

In [59]:
def info_gain(data, feature):
    examples= data.shape[0]

    DATA= divide_data(data,feature)
    keys= DATA.keys()

    ent_of_children= 0.0

    for key in keys:
        ent_of_children += ((DATA[key]['len']/examples) * entropy(DATA[key]['data']['Play']))

    info_gain= entropy(data['Play'])-ent_of_children
    return info_gain    


In [60]:
info_gain(data, 'Outlook')

0.24674981977443933

In [45]:
import math

2/3 * math.log2(2/3) + 1/3 * math.log2(1/3)

-0.9182958340544896

In [46]:
0.97-(2/5)-0.918

-0.3480000000000001

In [65]:
# Building a Decision Tree
import numpy as np
import pandas as pd

class DecisionTree:

    def __init__(self, depth=0, max_depth=5):

        self.children= {}
        self.key= None
        self.max_depth= max_depth
        self.depth= depth
        self.target= None
        #Prediction at any node

    def train(self, data):
        features= ['Outlook', 'Temperature', 'Humidity', 'Windy'] 

        info_gains= []

        for f in features:
            i_gain= info_gain(data, f)
            info_gains.append(i_gain) 

        self.key= features[np.argmax(info_gains)]
        #index of max value

        #Split the data

        Data_dict= divide_data(data, self.key)
        labels= list(data['Play'].value_counts().index)
        freq= list(data['Play'].value_counts().values)

        self.target= labels[np.argmax(freq)]

        ### Stoping conditions ###
        have_data= 0
        keys= Data_dict.keys()
        # If it is pure node
        for key in keys:
            if Data_dict[key]['len']>0:
                have_data+=1
        if have_data<2:
            return          

        # If max limit reached

        if self.depth>= self.max_depth:
            return 
        
        # Recursively call

        print("\t"*self.depth + "Making tree with - ", self.key)
        
        for key in keys:
            new_data= Data_dict[key]['data']
            self.children[key]= DecisionTree(depth=self.depth+1)
            self.children[key].train(new_data)

        return  

    def predict(self, test):
      if self.children=={}:
        return self.target
      return self.children[test[self.key][0]].predict(test) 


# Exploring Model
model= DecisionTree()
model.train(data)
        

Making tree with -  Outlook
	Making tree with -  Humidity
	Making tree with -  Windy


In [67]:
x_test= pd.DataFrame([['sunny','hot','high', False]], columns=list(data.columns.values[:-1]))
print(x_test)

model.predict(x_test)

  Outlook Temperature Humidity  Windy
0   sunny         hot     high  False


'no'

In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import warnings

# Load the golf dataset
data = pd.read_csv('golf_df.csv')

# Encode categorical variables
label_encoders = {}
for column in data.columns:
    if data[column].dtype == 'object':
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le  # Store the label encoder for future use

# Separate features and target variable
X = data.drop('Play', axis=1)
y = data['Play']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree Classifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Prepare test data (ensure it matches the trained feature set)
test_data = pd.DataFrame([[1, 1, 0, 0]], columns=X.columns) 
y_pred= model.predict(test_data)

# Decode the prediction to 'yes' or 'no'
prediction_result = 'yes' if y_pred[0] == 1 else 'no'
print(f"Prediction: {prediction_result}")


Prediction: yes
